# 🚀 Real CUDA Fluid Simulation

This notebook actually **compiles and runs the CUDA code** from the repository!
We'll build the fluidsGL executable and run the real CUDA fluid simulation.

**Requirements:** This needs a system with:
- NVIDIA GPU
- CUDA toolkit installed
- OpenGL/GLUT development libraries

If you're on Google Colab, make sure to select GPU runtime!

**Note:** If you encounter PTX toolchain errors, this notebook includes compatibility fixes.

In [ ]:
# First, let's check our environment
import os
import subprocess

def run_command(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        return result.returncode == 0, result.stdout, result.stderr
    except Exception as e:
        return False, "", str(e)

print("🔍 Environment Check")
print("=" * 50)

# Check if we have GPU
success, stdout, stderr = run_command("nvidia-smi")
if success:
    print("✅ GPU detected!")
    print(stdout.split('\n')[0])  # First line with driver info
    
    # Extract GPU name and compute capability
    gpu_info = run_command("nvidia-smi --query-gpu=name,compute_cap --format=csv,noheader,nounits")
    if gpu_info[0]:
        print(f"🔥 GPU Details: {gpu_info[1].strip()}")
else:
    print("❌ No GPU detected or nvidia-smi not available")
    print("   You'll need an NVIDIA GPU to run this efficiently")

# Check Python environment
print(f"\n🐍 Python: {os.sys.version}")
print(f"📁 Working directory: {os.getcwd()}")

print("\n" + "=" * 50)

In [ ]:
# Clone or navigate to the repository
repo_url = "https://github.com/tarun-bandi/cuda-fluid-sim.git"
repo_dir = "cuda-fluid-sim"

if os.path.exists(repo_dir):
    print(f"📁 Directory {repo_dir} already exists")
    os.chdir(repo_dir)
    !git pull  # Update if needed
else:
    print(f"📥 Cloning {repo_url}...")
    !git clone {repo_url}
    os.chdir(repo_dir)

print(f"\n📋 Repository contents:")
!ls -la

print(f"\n🔍 Source files:")
!find . -name '*.cu' -o -name '*.cpp' -o -name '*.cuh' | head -10

## 🛠️ Install Dependencies

Let's install the required tools and libraries:

In [ ]:
# Install CUDA and development tools
print("🔧 Installing dependencies...")

# Update package list
!apt-get update -qq

# Install CUDA toolkit and build tools
!apt-get install -y -qq nvidia-cuda-toolkit
!apt-get install -y -qq cmake build-essential

# Install OpenGL/GLUT for graphics
!apt-get install -y -qq freeglut3-dev libglew-dev libglu1-mesa-dev

# Install virtual framebuffer for headless OpenGL
!apt-get install -y -qq xvfb

print("\n✅ Dependencies installed!")

# Check CUDA installation and version
print("\n🔍 CUDA Version:")
success, stdout, stderr = run_command("nvcc --version")
if success:
    print(stdout)
    # Extract CUDA version for compatibility
    for line in stdout.split('\n'):
        if 'release' in line:
            print(f"🎯 CUDA Release: {line.strip()}")
            break
else:
    print("❌ CUDA compiler not found!")
    print(stderr)

## 🔧 Fix PTX Toolchain Compatibility

Let's check GPU compute capability and create compatible CUDA code:

In [ ]:
# Get detailed GPU information for compatibility
print("🔍 GPU Compute Capability Analysis:")
print("=" * 40)

# Query GPU compute capability
success, gpu_info, _ = run_command("nvidia-smi --query-gpu=name,compute_cap --format=csv,noheader,nounits")
if success:
    gpu_name, compute_cap = gpu_info.strip().split(', ')
    print(f"🔥 GPU: {gpu_name}")
    print(f"🧮 Compute Capability: {compute_cap}")
    
    # Determine appropriate CUDA architecture
    compute_major = int(compute_cap.split('.')[0])
    compute_minor = int(compute_cap.split('.')[1])
    
    if compute_major >= 7:
        arch_flag = "sm_70"
        print("✅ Modern GPU - using sm_70 architecture")
    elif compute_major >= 6:
        arch_flag = "sm_60"
        print("✅ Pascal GPU - using sm_60 architecture")
    elif compute_major >= 5:
        arch_flag = "sm_50"
        print("✅ Maxwell GPU - using sm_50 architecture")
    else:
        arch_flag = "sm_35"
        print("⚠️  Older GPU - using sm_35 architecture")
    
    print(f"🎯 Selected architecture: {arch_flag}")
else:
    print("❌ Could not determine GPU compute capability")
    arch_flag = "sm_60"  # Safe default
    print(f"🎯 Using default architecture: {arch_flag}")

print("\n" + "=" * 40)

## 📝 Examine the CUDA Source Code

Let's look at the actual CUDA kernels:

In [ ]:
# Display the CUDA kernel source
kernel_file = "src/fluidsGL_kernels.cu"
main_file = "src/fluidsGL.cpp"

if os.path.exists(kernel_file):
    print("🔥 CUDA Kernel File:")
    print("=" * 50)
    with open(kernel_file, 'r') as f:
        content = f.read()
        # Show first part of the file
        lines = content.split('\n')
        for i, line in enumerate(lines[:50]):
            print(f"{i+1:3d}: {line}")
        if len(lines) > 50:
            print(f"... ({len(lines) - 50} more lines)")
else:
    print(f"❌ {kernel_file} not found!")
    print("Available files:")
    !ls -la src/

print("\n" + "=" * 50)
print(f"📊 Code Statistics:")
!wc -l src/*.cu src/*.cpp 2>/dev/null || echo "Source files not found"

## 🔨 Build with Compatibility Fixes

Now let's compile with the correct architecture flags:

In [ ]:
# Create and enter build directory
build_dir = "build"
if os.path.exists(build_dir):
    import shutil
    shutil.rmtree(build_dir)
    
os.makedirs(build_dir)
os.chdir(build_dir)

print("🔨 Building CUDA Fluid Simulation with Compatibility Fixes")
print("=" * 60)

# Create a compatibility-focused CMakeLists.txt
cmake_content = f'''cmake_minimum_required(VERSION 3.8)
project(cuda_fluid_sim LANGUAGES CUDA CXX)

# Find CUDA
find_package(CUDA REQUIRED)
enable_language(CUDA)

# Set CUDA architecture for detected GPU
set(CMAKE_CUDA_FLAGS "${{CMAKE_CUDA_FLAGS}} -arch={arch_flag}")
set(CMAKE_CUDA_FLAGS "${{CMAKE_CUDA_FLAGS}} --use_fast_math")
set(CMAKE_CUDA_FLAGS "${{CMAKE_CUDA_FLAGS}} -Xcompiler -fPIC")

# Create simple compatibility test
add_executable(cuda_simple_test
    ../src/simple_cuda_test.cu
)

set_property(TARGET cuda_simple_test PROPERTY CUDA_SEPARABLE_COMPILATION ON)
target_link_libraries(cuda_simple_test ${{CUDA_LIBRARIES}})

# Try to build main fluid simulation if OpenGL is available
find_package(OpenGL QUIET)
find_package(GLUT QUIET)

if(OPENGL_FOUND AND GLUT_FOUND)
    add_executable(fluidsGL
        ../src/fluidsGL.cpp
        ../src/fluidsGL_kernels.cu
    )
    
    set_property(TARGET fluidsGL PROPERTY CUDA_SEPARABLE_COMPILATION ON)
    target_link_libraries(fluidsGL 
        ${{CUDA_LIBRARIES}} 
        ${{OPENGL_LIBRARIES}} 
        ${{GLUT_LIBRARY}}
    )
    
    message(STATUS "Building full OpenGL fluid simulation")
else()
    message(STATUS "OpenGL/GLUT not found - building CUDA test only")
endif()
'''

with open("CMakeLists.txt", "w") as f:
    f.write(cmake_content)

print(f"✅ Created CMakeLists.txt with architecture: {arch_flag}")

# Configure with CMake
print("\n📋 Step 1: CMake Configuration")
cmake_success, cmake_out, cmake_err = run_command("cmake ..")

if cmake_success:
    print("✅ CMake configuration successful!")
    if cmake_out:
        # Show relevant output
        for line in cmake_out.split('\n'):
            if 'CUDA' in line or 'OpenGL' in line or 'Building' in line:
                print(f"   {line}")
else:
    print("❌ CMake configuration failed!")
    print("Error output:")
    print(cmake_err[:1000])

print("\n" + "=" * 30)

In [ ]:
# Build the project
print("🔨 Step 2: Compilation")

if cmake_success:
    build_success, build_out, build_err = run_command("make -j4")
    
    if build_success:
        print("✅ Build successful!")
        print("\n📁 Build artifacts:")
        !ls -la
        
        # Look for executables
        executables = [f for f in os.listdir('.') if os.access(f, os.X_OK) and not f.startswith('.')]
        if executables:
            print(f"\n🎯 Found executables: {', '.join(executables)}")
        else:
            print("\n❌ No executables found after build")
            
    else:
        print("❌ Build failed!")
        print("Build output:")
        print(build_out[:1500] if build_out else "No output")
        print("\nBuild errors:")
        print(build_err[:1500] if build_err else "No errors")
        
        # Try direct CUDA compilation with specific arch
        print(f"\n🔧 Trying direct CUDA compilation with {arch_flag}...")
        direct_cmd = f"nvcc -arch={arch_flag} -o simple_test ../src/simple_cuda_test.cu"
        direct_success, direct_out, direct_err = run_command(direct_cmd)
        
        if direct_success:
            print("✅ Direct CUDA compilation successful!")
            !ls -la simple_test
        else:
            print("❌ Direct CUDA compilation also failed")
            print(direct_err[:500])
else:
    print("❌ Skipping build due to CMake failure")

print("\n" + "=" * 30)

## 🚀 Test CUDA Compatibility

Let's run the compatibility test to verify the PTX issue is fixed:

In [ ]:
# Run the compatibility test
print("🧪 Running CUDA Compatibility Test")
print("=" * 40)

# Look for executables
test_executables = ['cuda_simple_test', 'simple_test']
found_test = None

for exe in test_executables:
    if os.path.exists(exe) and os.access(exe, os.X_OK):
        found_test = exe
        break

if found_test:
    print(f"✅ Found test executable: {found_test}")
    
    # Run the test
    print(f"\n🚀 Running {found_test}...")
    test_success, test_out, test_err = run_command(f"./{found_test}")
    
    if test_success:
        print("✅ CUDA test completed successfully!")
        print("Output:")
        print(test_out)
        
        if "PTX" in test_err or "toolchain" in test_err:
            print("⚠️  PTX warnings detected:")
            print(test_err)
        elif test_err:
            print("ℹ️  Additional info:")
            print(test_err)
    else:
        print("❌ CUDA test failed!")
        print("Error output:")
        print(test_err)
        
        if "PTX" in test_err or "toolchain" in test_err:
            print("\n🔧 PTX Toolchain Issue Detected!")
            print("This usually means:")
            print("1. CUDA toolkit version mismatch with GPU driver")
            print("2. Wrong architecture flag for your GPU")
            print(f"3. Try using older CUDA toolkit or update GPU drivers")
else:
    print("❌ No test executable found!")
    print("Available files:")
    !ls -la

print("\n" + "=" * 40)

## 🎯 Run the Main Fluid Simulation

If everything works, let's try the main application:

In [ ]:
# Look for the main fluid simulation executable
if os.path.exists('fluidsGL') and os.access('fluidsGL', os.X_OK):
    print("🎯 Found main fluid simulation: fluidsGL")
    !ls -la fluidsGL
    
    print("\n🚀 Attempting to run CUDA Fluid Simulation...")
    print("Note: This requires display/GUI. In headless mode, it may show initialization info only.")
    
    # Try to run with timeout
    print("\nRunning with 10-second timeout...")
    run_success, run_out, run_err = run_command("timeout 10s ./fluidsGL || echo 'Simulation requires display'")
    
    if run_out:
        print("📋 Output:")
        print(run_out)
    
    if run_err:
        print("📋 Messages:")
        print(run_err)
    
    if "CUDA" in run_out or "GPU" in run_out:
        print("\n✅ CUDA fluid simulation is working!")
        print("🎮 To run locally: cd build && ./fluidsGL")
    else:
        print("\n⚠️  Simulation needs display environment to run fully")
else:
    print("ℹ️  Main fluid simulation not built (likely due to missing OpenGL)")
    print("   This is normal in headless environments")
    
    if found_test:
        print(f"\n✅ But CUDA is working as demonstrated by {found_test}!")
        print("🏠 For full graphics, run locally with GPU and display")

print("\n" + "=" * 50)

## 💻 For Local Development

To run this simulation on your local machine with proper GPU acceleration:

### Prerequisites:
1. **NVIDIA GPU** with CUDA support
2. **CUDA Toolkit** installed
3. **Development tools**: CMake, C++ compiler
4. **Graphics libraries**: OpenGL, GLUT

### Installation Commands:

**Ubuntu/Debian:**
```bash
sudo apt update
sudo apt install nvidia-cuda-toolkit cmake build-essential
sudo apt install freeglut3-dev libglew-dev libglu1-mesa-dev
```

**macOS:**
```bash
# Install CUDA toolkit from NVIDIA website
# Install Xcode command line tools
xcode-select --install
# OpenGL/GLUT should be available via system frameworks
```

### Build and Run:
```bash
git clone https://github.com/tarun-bandi/cuda-fluid-sim.git
cd cuda-fluid-sim
# Use the provided test script:
./test_cuda_build.sh
# OR manual build:
mkdir build && cd build
cmake ..
make -j4
./fluidsGL
```

### Controls:
- **Click and drag**: Add fluid density and heat
- **R key**: Reset simulation
- **ESC**: Exit

## 🔧 Troubleshooting PTX Errors

If you encounter **"unsupported toolchain"** errors:

1. **Check GPU Compute Capability**:
   ```bash
   nvidia-smi --query-gpu=compute_cap --format=csv,noheader,nounits
   ```

2. **Use Appropriate Architecture Flag**:
   - Compute 7.x: `-arch=sm_70`
   - Compute 6.x: `-arch=sm_60`  
   - Compute 5.x: `-arch=sm_50`
   - Compute 3.x: `-arch=sm_35`

3. **Update Drivers or CUDA**:
   - Ensure CUDA toolkit matches your GPU driver version
   - Consider using CUDA 11.x for better compatibility

## 🔬 What This Demonstrates

This project showcases real **GPU-accelerated fluid dynamics** using:

1. **CUDA Kernels**: 
   - `advectKernel`: Particle advection with bilinear interpolation
   - `diffuseKernel`: Viscosity and heat diffusion
   - `addForceKernel`: Buoyancy forces
   - `projectionKernels`: Pressure projection for incompressible flow

2. **Advanced GPU Techniques**:
   - Texture memory for optimized access patterns
   - Efficient grid/block dimensions
   - Device memory management
   - Kernel synchronization
   - Architecture-specific optimization

3. **Real-time Visualization**:
   - OpenGL rendering
   - Interactive mouse controls
   - Color-coded visualization of density and temperature

4. **Production Practices**:
   - Cross-platform compatibility
   - Robust error handling
   - Professional build system
   - GPU architecture detection

This is **production-quality CUDA code** with real-world compatibility handling! 🚀